## Simulation

In [1]:
import sqlite3
import simpy
import Dijkstra

# Klasse Produkt definieren - Attribute siehe DB
class Produkt:
    def __init__(self, id, bezeichnung, dlz_plan, losgroesse, bestand_rtl, bestand_ftl):
        self.id = id
        self.bezeichnung = bezeichnung
        self.dlz_plan = dlz_plan
        self.losgroesse = losgroesse
        self.bestand_rtl = bestand_rtl
        self.bestand_ftl = bestand_ftl        

# Überschreiben der __repr__ Methode, damit die Objekte als Klartext beim Printen angezeigt werden
    def __repr__(self):
        return (f"Produkt(id={self.id}, bezeichnung='{self.bezeichnung}', dlz_plan={self.dlz_plan}, "
                f"losgroesse={self.losgroesse}, bestand_rtl={self.bestand_rtl}, bestand_ftl={self.bestand_ftl})")

# Klasse Auftrag definieren - Attribute siehe DB
class Auftrag:
    def __init__(self, charge, id, menge):
        self.charge = charge
        self.id = id
        self.arbeitsplan = []
        self.menge = menge
    
    # Überschreiben der __repr__ Methode, damit die Objekte als Klartext beim Printen angezeigt werden
    def __repr__(self):
        return f"Auftrag(charge={self.charge}, id={self.id}, arbeitsplan={self.arbeitsplan})"

# Klasse FFZ definieren - Attribute siehe DB
class FFZ:
    def __init__(self, id, speed, akkulaufzeit):
        self.id = id
        self.speed = speed
        self.akkulaufzeit = akkulaufzeit
        self.curr_transport = None
        self.queue = []
    
    # Überschreiben der __repr__ Methode, damit die Objekte als Klartext beim Printen angezeigt werden
    def __repr__(self):
        return f"Auftrag(id={self.id})"
    
# Klasse Factory definieren
class Factory:
    def __init__(self, env, db_path):
        self.env = env # Simulationsumgebung
        self.db_path = db_path # Pfad zur Datenbank
        self.db() 
        self.lade_produkte()
        self.lade_ffz()
        self.saegen_queue = []
        self.saegen_status = "v"
        # v - verfügbar, b - belegt, d - defekt
        self.drehen_queue = []
        self.drehen_1_status = "v"
        self.drehen_2_status = "v"
        self.pruefen_queue = []
        self.pruefen_status = "v"
        env.process(self.auftragsfreigabe(env))
        env.process(self.absatz(env))
        # Dijkstra
        self.graph = Dijkstra.graph([
        ("a", "b", 4), ("a", "c", 5), ("a", "d", 10),
        ("b", "c", 5), ("b", "d", 8), ("c", "d", 7),
        ("d", "e", 2)
    ])

    # DB Verbindung herstellen
    def db(self):
        self.conn = sqlite3.connect(self.db_path)
        self.cursor = self.conn.cursor()

    # Erstelle Produkt Objekte auf Basis der Produktstammdaten siehe db
    def lade_produkte(self):
        self.produkte = []

        # Produkte aus der Tabelle produktstammdaten abrufen
        self.cursor.execute('SELECT * FROM produktstammdaten')
        rows = self.cursor.fetchall()

        # Für jede Zeile ein Produkt-Objekt erstellen
        for row in rows:
            produkt = Produkt(id=row[0], bezeichnung=row[1], dlz_plan=row[2], losgroesse=row[3], bestand_rtl=row[4], bestand_ftl=row[5])
            self.produkte.append(produkt)
            # Name des Objekts entspricht der Bezeichnung
            globals()[produkt.bezeichnung] = produkt

    def lade_ffz(self):
        self.ffzs = []

        self.cursor.execute('SELECT * FROM ffz')
        rows = self.cursor.fetchall()

        for row in rows:
            ffz = FFZ(id=row[0], speed=row[1], akkulaufzeit=row[2])
            self.ffzs.append(ffz)    

    # Finde Source und Destination des Transports
    def find_source_destination(self, arbeitsschritt, folgeschritt):
        # Dictionary mit den Standorten der Arbeitsstationen
        location_map = {
            'SAE': 'b',  # Sägen an Standort B
            'DRH': 'c',  # Drehen an Standort C
            'QPR': 'd',  # Prüfen an Standort D
        }

        # Bestimme den Standort für den aktuellen Arbeitsschritt
        if arbeitsschritt is not None:
            source = location_map.get(arbeitsschritt[0])
        else: 
            source = 'a'

        if folgeschritt is not None:
            destination = location_map.get(folgeschritt[0])
        else:
            destination = 'e'

        return source, destination


    # Einfachster Fall Bestellbestands-Verfahren - Freigabe neuer Aufträge bei Unterschreitung der Bestellbestands
    def auftragsfreigabe(self, env):
        self.auftraege = []
        
        while True: # Freigabe Prüflauf läuft kontinuierlich
            for produkt in self.produkte:
                # Berechne die Summe der Mengen aller Aufträge für das Produkt (Lagerbestand + Summe offene Aufträge)
                offene_auftragsmenge = sum(auftrag.menge for auftrag in self.auftraege if auftrag.id == produkt.id)
                
                while (produkt.bestand_ftl + offene_auftragsmenge) < 2200:
                    # Auftrag in die Datenbank einfügen
                    self.cursor.execute('INSERT INTO auftraege (id) VALUES (?)', (produkt.id,))

                    # Abrufen der Charge für den letzten eingefügten Auftrag
                    self.cursor.execute('SELECT charge FROM auftraege ORDER BY charge DESC LIMIT 1')
                    charge = self.cursor.fetchone()[0]  

                    # Erstellen eines Auftrag-Objekts
                    auftrag = Auftrag(charge=charge, id=produkt.id, menge=produkt.losgroesse)
                    
                    # Suche nach den Arbeitsschritten
                    self.cursor.execute('SELECT bmg, t_plan, nr FROM arbeitsplaene WHERE id=? ORDER BY nr ASC', (produkt.id,))
                    rows = self.cursor.fetchall()  
                    
                    # Alle Arbeitsplan-Einträge zum Auftrag hinzufügen
                    for row in rows:
                        auftrag.arbeitsplan.append(row) 

                    # Speichern des Auftrag-Objekts in der Liste
                    self.auftraege.append(auftrag)
                  
                    # Auftrag freigeben und als Prozess starten
                    env.process(self.produktion(auftrag, produkt,env))

                    # Commit
                    self.conn.commit()

                    # Aktualisiere die offene Auftragsmenge für die nächste Schleife
                    offene_auftragsmenge = sum(auftrag.menge for auftrag in self.auftraege if auftrag.id == produkt.id)

                    yield env.timeout(0)
            yield env.timeout(1)         
    
    # Produktionsfunktion zum Durchlauf aller Arbeitsschritte 
    def produktion(self, auftrag, produkt, env):
        """Führt die Produktionsschritte für einen gegebenen Auftrag durch.
        Args:
        auftrag: Das Auftrag-Objekt, das die Arbeitspläne enthält.
        produkt: Das Produkt, das bearbeitet wird.
        env: Die SimPy-Umgebung, die die Simulation verwaltet.
        """
        workstations = {
            'SAE': self.saegen,
            'DRH': self.drehen,
            'QPR': self.pruefen,
        }

        for i, arbeitsschritt in enumerate(auftrag.arbeitsplan):
            station = arbeitsschritt[0]
            if station in workstations:
                if arbeitsschritt == auftrag.arbeitsplan[0]:
                    # Bestimme Source und Destination
                    source, destination = self.find_source_destination(None, arbeitsschritt)
                    yield env.process(self.transport(auftrag, source, destination, env))
                    print(f"Transport von {source} nach {destination} für Auftrag {auftrag.charge} zu Beginn")


                yield env.process(workstations[station](auftrag, arbeitsschritt, env))

                # Nächster Arbeitsschritt (falls vorhanden)
                folgeschritt = auftrag.arbeitsplan[i + 1] if i + 1 < len(auftrag.arbeitsplan) else None

                # Bestimme Source und Destination
                source, destination = self.find_source_destination(arbeitsschritt, folgeschritt)
                yield env.process(self.transport(auftrag, source, destination, env))
                print(f"Transport von {source} nach {destination} für Auftrag {auftrag.charge} nach Schritt {arbeitsschritt[0]}")

            else:
                raise ValueError(f"Unbekannter Arbeitsschritt: {station}")

            # Falls der letzte Arbeitsschritt ausgeführt wurde:
            if arbeitsschritt == auftrag.arbeitsplan[-1]:
                # Auftrag aus Auftragsliste entfernen
                self.auftraege.remove(auftrag)
                    
                # Bestand um die Losgröße erhöhen
                produkt.bestand_ftl += produkt.losgroesse

                # Bestand des Produkts in der Datenbank aktualisieren
                self.cursor.execute('UPDATE produktstammdaten SET bestand_ftl = ? WHERE id = ?', (produkt.bestand_ftl, produkt.id))
                self.conn.commit()
          

    ##### Produktionsfunktionen
    def saegen(self, auftrag, arbeitsschritt, env):
        # Auftrag zur Queue der Arbeitsstation hinzufügen
        self.saegen_queue.append(auftrag)

        # Prüfen ob die Maschine belegt ist
        while self.saegen_status == ("b" or "d"):
            yield env.timeout(1)
        
        # Status belegt, bei Start der Bearbeitung
        self.saegen_status = "b"

        print(f"Start Sägen: {env.now}, Auftrag {auftrag.charge}")
        yield env.timeout(arbeitsschritt[1])  # Bearbeitungszeit
        print(f"Ende Sägen: {env.now}, Auftrag {auftrag.charge} abgeschlossen")

        # Status verfügbar, bei Ende der Bearbeitung
        self.saegen_status = "v"

        # Entferne den Auftrag nach der Bearbeitung aus der Warteschlange
        self.saegen_queue.remove(auftrag)

    def drehen(self, auftrag, arbeitsschritt, env):
        # Aufbau der weiteren Produktionsfunktionen analog der ersten
        # Besonderheit beim Drehen: 2 Arbeitssysteme        
        
        self.drehen_queue.append(auftrag)

        while self.drehen_1_status == ("b" or "d") and self.drehen_2_status == ("b" or "d"):
            yield env.timeout(1)
        
        if self.drehen_1_status == "v":

            self.drehen_1_status = "b"

            print(f"Start Drehen auf DRH1: {env.now}, Auftrag {auftrag.charge}")
            yield env.timeout(arbeitsschritt[1])  # Nutze die Zeit aus dem Arbeitsschritt
            print(f"Ende Drehen auf DRH1: {env.now}, Auftrag {auftrag.charge} abgeschlossen")

            self.drehen_1_status = "v"

            # Entferne den Auftrag nach der Bearbeitung aus der Warteschlange
            self.drehen_queue.remove(auftrag)

        elif self.drehen_2_status == "v":

            self.drehen_2_status = "b"

            print(f"Start Drehen auf DRH2: {env.now}, Auftrag {auftrag.charge}")
            yield env.timeout(arbeitsschritt[1])  # Nutze die Zeit aus dem Arbeitsschritt
            print(f"Ende Drehen auf DRH2: {env.now}, Auftrag {auftrag.charge} abgeschlossen")

            self.drehen_2_status = "v"

            # Entferne den Auftrag nach der Bearbeitung aus der Warteschlange
            self.drehen_queue.remove(auftrag)

    def pruefen(self, auftrag, arbeitsschritt, env):
        self.pruefen_queue.append(auftrag)

        while self.pruefen_status == ("b" or "d"):
            yield env.timeout(1)
        
        self.pruefen_status = "b"

        print(f"Start Prüfen: {env.now}, Auftrag {auftrag.charge}")
        yield env.timeout(arbeitsschritt[1])  # Nutze die Zeit aus dem Arbeitsschritt
        print(f"Ende Prüfen: {env.now}, Auftrag {auftrag.charge} abgeschlossen")

        self.pruefen_status = "v"

        # Entferne den Auftrag nach der Bearbeitung aus der Warteschlange
        self.pruefen_queue.remove(auftrag)

    ##### Transportfunktionen
    def transport(self, auftrag, source, destination, env):
        # Berechnung der Distance mit Dijkstra Algorithmus
        distance = self.graph.dijkstra(source, destination)[1] # self.graph.dijkstra("a", "d")[0] für gewählte Route
        
        status = False

        # Fortlaufende Überprüfung ob FFZ verfügbar ist
        while status == False:
            for ffz in self.ffzs:
                if ffz.curr_transport == None:
                    
                    # Zuweisen des Auftrags zu FFZ
                    ffz.curr_transport = auftrag
                    
                    # Berechnung der Fahrdauer
                    duration = distance * 10 / ffz.speed
                    
                    # Abwarten der Fahrtdauer
                    yield env.timeout(duration)
                    print(f"transport mit Fahrzeug: {ffz.id}")

                    # Zurücksetzen der Stati
                    ffz.curr_transport = None
                    status = True

                    break
                else:
                    continue
            yield env.timeout(1)

    def absatz(self, env):
        while True:
            for produkt in self.produkte:
                produkt.bestand_ftl -= 100
            
            yield env.timeout(150)


In [2]:
env = simpy.Environment()

#Factory-Instanz erstellen und die Datenbank laden
factory = Factory(env, 'prod_data.db')
env.run(until=1000)

transport mit Fahrzeug: F2
Transport von a nach b für Auftrag 20000001 zu Beginn
Start Sägen: 3.857142857142857, Auftrag 20000001
transport mit Fahrzeug: F1
Transport von a nach b für Auftrag 20000000 zu Beginn
transport mit Fahrzeug: F2
Transport von a nach b für Auftrag 20000002 zu Beginn
transport mit Fahrzeug: F1
transport mit Fahrzeug: F2
Transport von a nach b für Auftrag 20000003 zu Beginn
Transport von a nach b für Auftrag 20000004 zu Beginn
transport mit Fahrzeug: F2
transport mit Fahrzeug: F1
Transport von a nach b für Auftrag 20000006 zu Beginn
Transport von a nach b für Auftrag 20000005 zu Beginn
transport mit Fahrzeug: F1
Transport von a nach b für Auftrag 20000007 zu Beginn
Ende Sägen: 73.85714285714286, Auftrag 20000001 abgeschlossen
Start Sägen: 73.85714285714286, Auftrag 20000002
transport mit Fahrzeug: F1
Transport von b nach c für Auftrag 20000001 nach Schritt SAE
Start Drehen auf DRH1: 79.85714285714286, Auftrag 20000001
Ende Drehen auf DRH1: 129.85714285714286, Auf

In [13]:
# Verbindung zur bestehenden Datenbank herstellen
conn = sqlite3.connect('prod_data.db')
cursor = conn.cursor()

# Abfrage, um alle Tabellen in der Datenbank anzuzeigen
cursor.execute('''
    SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';
''')

# Alle Tabellen auflisten
tables = cursor.fetchall()

# Für jede Tabelle den Inhalt anzeigen
for table in tables:
    table_name = table[0]
    print(f"Inhalt der Tabelle: {table_name}")
    
    # Inhalt der Tabelle abfragen
    cursor.execute(f'SELECT * FROM {table_name}')
    rows = cursor.fetchall()
    
    # Spaltennamen der Tabelle anzeigen
    column_names = [description[0] for description in cursor.description]
    print(column_names)  # Spaltennamen drucken
    
    # Inhalt der Tabelle Zeile für Zeile drucken
    for row in rows:
        print(row)
    print("\n" + "-"*50 + "\n")  # Trennlinie für bessere Lesbarkeit

# Verbindung schließen
conn.close()

Inhalt der Tabelle: produktstammdaten
['id', 'bezeichnung', 'dlz_plan', 'losgroesse', 'Bestand_RTL', 'Bestand_FTL']
('A2', 'Deckel', 150, 50, 1000, 2200)
('A7', 'Welle', 50, 200, 1000, 2200)

--------------------------------------------------

Inhalt der Tabelle: auftraege
['Charge', 'id']
(20000000, 'A2')
(20000001, 'A2')
(20000002, 'A2')
(20000003, 'A2')
(20000004, 'A2')
(20000005, 'A2')
(20000006, 'A7')
(20000007, 'A7')
(20000008, 'A2')
(20000009, 'A2')
(20000010, 'A2')
(20000011, 'A2')
(20000012, 'A7')
(20000013, 'A2')
(20000014, 'A2')
(20000015, 'A2')
(20000016, 'A2')
(20000017, 'A7')
(20000018, 'A2')
(20000019, 'A2')
(20000020, 'A2')
(20000021, 'A2')
(20000022, 'A7')
(20000023, 'A2')
(20000024, 'A2')
(20000025, 'A2')
(20000026, 'A2')
(20000027, 'A7')
(20000028, 'A2')
(20000029, 'A2')
(20000030, 'A2')
(20000031, 'A2')
(20000032, 'A7')
(20000033, 'A2')
(20000034, 'A2')
(20000035, 'A2')
(20000036, 'A2')
(20000037, 'A7')
(20000038, 'A2')
(20000039, 'A2')
(20000040, 'A2')
(20000041, '

#### to dos
- produktion db umbennen in aufträge
- FK BMG tabelle
- NR- für AS hinzufügen
- Freigabelogik anpassen hinsichtlich der Bestnadserhöhung
- Mindestbestände pflegen